# Демонстрация Spell Checker

В этом ноутбуке будет произведена демонстрация написанной модели на примере нескольких предложений.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import sys
import os
import re
from string import punctuation
sys.path.append('..')

import dotenv
import numpy as np
import pandas as pd
from transformers import BertForMaskedLM, BertTokenizer, BertConfig

from deeppavlov.models.spelling_correction.levenshtein import (
    LevenshteinSearcherComponent
)
from deeppavlov.core.data.simple_vocab import SimpleVocabulary

import kenlm
from sacremoses import MosesTokenizer, MosesDetokenizer

from src.models.SpellChecker.levenshtein_candidate_generator import (
    LevenshteinCandidateGenerator
)
from src.models.SpellChecker.kenlm_position_selector import (
    KenlmPositionSelector
)
from src.models.BertScorer.bert_scorer_correction import (
    BertScorerCorrection
)
from src.models.SpellChecker.bert_candidate_scorer import (
    BertCandidateScorer
)
from src.models.SpellChecker.margin_stopping_criteria import (
    MultiplicativeMarginStoppingCriteria
)
from src.models.SpellChecker.spell_checker import IterativeSpellChecker

from IPython.display import display
from tqdm.notebook import tqdm

[nltk_data] Downloading package punkt to /home/mrgeekman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/mrgeekman/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [3]:
PROJECT_PATH = os.path.join(os.path.abspath(''), os.pardir)
CONFIGS_PATH = os.path.join(PROJECT_PATH, 'src', 'configs')
os.environ['DP_PROJECT_PATH'] = PROJECT_PATH

In [4]:
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
MODEL_PATH = os.path.join(PROJECT_PATH, 'models')

## Инициализация

Начнем с того, что инициализируем все необходимые компоненты модели. Параллельно так же будет описана роль каждого компонента в системе.

### Tokenizer/Detokenizer

Этот компонент отвечает за токенизаци/детокенизацию исходного предложения. В качестве основы было решено взять токенизатор из библиотеки [sacremoses](https://github.com/alvations/sacremoses).

In [5]:
raw_tokenizer = MosesTokenizer(lang='ru')
raw_detokenizer = MosesDetokenizer(lang='ru')
tokenizer = lambda x: raw_tokenizer.tokenize(x)
detokenizer = lambda x: raw_detokenizer.detokenize(x)

### Candidate Generator

Этот компонент отвечает за то, чтобы генерировать кандидатов для каждой позиции токенизированного предложения. 

На данный момент берутся слова из словаря на заданном расстоянии Дамерау-Левенштейна от исходного токена. Иногда эти слова еще разбиваются пробелами.

В качестве словаря был взят [этот](https://github.com/danakt/russian-words/).

In [6]:
vocab_path = os.path.join(DATA_PATH, 'external', 'russian_words', 
                          'russian_words_vocab.dict')
vocab = SimpleVocabulary(load_path=vocab_path, save_path=vocab_path)
levenshtein_searcher_component = LevenshteinSearcherComponent(
    words=vocab.keys(), max_distance=1
)
candidate_generator = LevenshteinCandidateGenerator(
    levenshtein_searcher_component
)

2021-01-16 18:51:07.477 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/mrgeekman/Documents/MIPT/НИР/Repo/data/external/russian_words/russian_words_vocab.dict]


### Position Selector

Этот компонент отвечает за нахождение оптимальной позиции для замены и предварительный выбор кандидатов для этой позиции.

На данный момент берутся две языковые модели: слева-направо и справа-налево, которые работают с текстом без пунктуации. Для каждой позиции по просматривается весь список кандидатов при использовании левого и правого контекстов и подсчитывается log-prob. Если токен состоит из нескольких подтокенов (например, если в слове есть пробел), то скор суммируется.

Так как для каждого кандидата имеется два log-prob их результат аггрегируется в некий более удобный положительный скор. На данный момент от каждого log-prob берется функция $-1/x$ и от их результата считается среднее гармоническое.

Так же мы знаем какой из кандидатов в каждой позиции соответствует изначальному токену (он обязательно там есть) и скор каждого кандидата с ним сравнивается.

Выбранная позиция для изменения -- та, у которой наибольшее отношение максимального скора к скору текущего токена. На основе этого отношени так же отбирается предварительный список кандидатов.

In [7]:
model_left_right = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'left_right.arpa.binary')
)
model_right_left = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'right_left.arpa.binary')
)
position_selector = KenlmPositionSelector(model_left_right, model_right_left)

### Candidate Scorer

Этот компонент отвечает за выбор наилучшего кандидата из списка предложенных position selector.

На данный момент берется модель [Conversational RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html). В первую очередь токенизируется (WordPiece) исходное предложение с MASK-токеном на месте замяемого токена. Хочется просто запустить Masked Language Modeling и попробовать поподставлять кандидатов вместо MASK, но проблема в том, что некоторые кандидаты состоят из более, чем одного токена. В таком случае мы токенизируем всех кандидатов и пытаемся двигать MASK-токен по каждой позиции внутри него, делая другие позиции UNK-токеном и отключая для них attention (не обязательно ставить UNK-токен, это было сделано для удобства). Для аггрегации log-prob скоров внутри одного кандидата берется сумма. По аналогии с position selector к итоговому скору кандидата применяется $-1/x$.

В результате отбиратеся кандидат с наилучшим скором.

In [8]:
BERT_PATH = os.path.join(MODEL_PATH, 'conversational_rubert')
config = BertConfig.from_json_file(
    os.path.join(BERT_PATH, 'bert_config.json')
)
model = BertForMaskedLM.from_pretrained(
    os.path.join(BERT_PATH, 'pytorch_model.bin'),
    config=config
)
bert_tokenizer = BertTokenizer(os.path.join(BERT_PATH, 'vocab.txt'))
scorer_basis = BertScorerCorrection(model, bert_tokenizer)
candidate_scorer = BertCandidateScorer(scorer_basis)

Some weights of the model checkpoint at /home/mrgeekman/Documents/MIPT/НИР/Repo/notebooks/../models/conversational_rubert/pytorch_model.bin were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at /home/mrgeekman/Documents/MIPT/НИР/Repo/notebooks/../models/conversational_rubert/pytorch_model.bin and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a d

### Stopping Criteria

Этот компонент отвечает за остановку итераций исправления.

На данный момент берутся скор текущего токена и лучший скор, полученный при помощи BERT. На основе их отношения решается делать следующую итерацию или нет.

Если отношение превышает некую константу, то мы продолжаем, иначе останавливаемся.

In [9]:
margin_constant = 1.01
stopping_criteria = MultiplicativeMarginStoppingCriteria(margin_constant)

### Spell Checker

Выше были описаны все компоненты модели. Итого, имеем этапы:
1. Генерация кандидатов
2. Итерации до тех пор, пока не сработает критерий останова или не исчерпается максимальное количество итераций
    * Поиск лучшей позиции для исправления и отбор кандидатов для исправления
    * Выбор лучшего исправления
    * Исправление текущего предложения
    * Проверка критерия останова

In [10]:
# количество кандидатов, отбираемых position selector
num_selected_candidates = 16
# максимальное количество итераций
max_it = 5

spellchecker = IterativeSpellChecker(
    candidate_generator,
    position_selector,
    candidate_scorer,
    stopping_criteria,
    tokenizer,
    detokenizer,
    num_selected_candidates,
    max_it
)

## Демонстрация

Возьмем 5 предложений из обучающей части датасета вместе с ответами и посмотрим на результаты.

In [11]:
num_examples = 5

with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 'train_source.txt'), 
    'r'
) as inf:
    all_sentences = inf.readlines()
    
with open(
    os.path.join(DATA_PATH, 'external', 'spell_ru_eval', 
                 'train_corrected.txt'), 
    'r'
) as inf:
    all_corrected_sentences = inf.readlines()

In [12]:
np.random.seed(42)
num_sentences = len(all_sentences)
all_indices = np.arange(num_sentences)
np.random.shuffle(all_indices)
indices = all_indices[:num_examples]

examples = [all_sentences[idx].strip() for idx in indices]
examples_true = [all_corrected_sentences[idx].strip() for idx in indices]

In [13]:
indices

array([1860,  353, 1333,  905, 1289])

In [14]:
examples

['Но стоит ли так быстро поддаваться унынию и пессимистическому настроению?',
 'Дооолго шли, солнце уже садится стало, в лесу слышны залпы - тревога в княжестве, ищут нас - беглянок.',
 'Ну все седня будут вещать про природные катаклизмы, свои отпуска ( хотя об этом и нельзя писать ) и прочий пазитив.',
 'Символизируэт стремление мужчин все в этой жизни делать ради женщин.',
 'С людьми уже давно развела жизнь или ты даже и не знаешь их совсем, а тут почти вся их жизнь - словно на ладони, будто интереснейший роман читаешь.']

In [15]:
examples_true

['Но стоит ли так быстро поддаваться унынию и пессимистическому настроению',
 'Долго шли солнце уже садиться стало в лесу слышны залпы тревога в княжестве ищут нас беглянок',
 'Ну все сегодня будут вещать про природные катаклизмы свои отпуска хотя об этом и нельзя писать и прочий позитив',
 'Символизирует стремление мужчин все в этой жизни делать ради женщин',
 'С людьми уже давно развела жизнь или ты даже и не знаешь их совсем а тут почти вся их жизнь словно на ладони будто интереснейший роман читаешь']

Запустим наш spell checker.

In [16]:
examples_corrected = spellchecker(examples)

In [22]:
for i in range(num_examples):
    print(f'Original:\t{examples[i]}')
    print(f'Corrected:\t{examples_corrected[i]}')
    print(f'True:\t\t{examples_true[i]}')
    print()

Original:	Но стоит ли так быстро поддаваться унынию и пессимистическому настроению?
Corrected:	но стоит ли так быстро поддаваться унынию и пессимистическому настроению
True:		Но стоит ли так быстро поддаваться унынию и пессимистическому настроению

Original:	Дооолго шли, солнце уже садится стало, в лесу слышны залпы - тревога в княжестве, ищут нас - беглянок.
Corrected:	дооолго ли солнце уже садится стало в лесу слышны залпы тревога в княжестве ищут нас беглянок
True:		Долго шли солнце уже садиться стало в лесу слышны залпы тревога в княжестве ищут нас беглянок

Original:	Ну все седня будут вещать про природные катаклизмы, свои отпуска ( хотя об этом и нельзя писать ) и прочий пазитив.
Corrected:	ну все седня будут вещать про природные катаклизмы свои отпуска хотя об этом и нельзя писать и прочий пазитив
True:		Ну все сегодня будут вещать про природные катаклизмы свои отпуска хотя об этом и нельзя писать и прочий позитив

Original:	Символизируэт стремление мужчин все в этой жизни делат

К сожалению, исправляется все весьма плохо.